# <h1 style="text-align: center; font-size: 36px; color: #3498db; font-weight: bold;">Prosody Tools </h1>
## <h2 style="text-align: center; font-size: 28px; color: #2ecc71; font-weight: bold;">Prosody Interface</h2>
### <h3 style="text-align: center; font-size: 24px; color: #e74c3c; font-family: 'Arial', sans-serif; font-weight: bold;">Prosody Active Learning</h3>


---
Project: Prosody

Institution: Reutlingen University

Author: Mohamad Eyad Alkostanini

Supervisor: Bakir Hadzic, Parvez Mohammed, (ViSiR)

Professor: Prof. Matthias Rätsch

Description:
This script performs diarization as part of the Prosody in Mental Health project.

--- 

In [1]:
##############
# Library
##############

from tkinter import *
import tkinter as tk
import pyaudio
import wave
import os 
import numpy as np
import pandas as pd
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
from pydub import AudioSegment
from pydub.playback import play
from datetime import datetime, timezone
from tkinter import messagebox, Radiobutton
import re
import shutil
import sounddevice as sd



In [2]:
###############################################
# loding the model, weights, scaler and encoder
###############################################

global PREDICTED_CLASS     
OUTPUT_FILE=r"./Output/input_voice.wav"
Output_folder= r"./Output"
if not os.path.exists(Output_folder):    
    os.makedirs(Output_folder)

In [3]:
#####################
# Functions section #
#####################################



from emotion2vecplus import Emotion2Vec
emotion2vec = Emotion2Vec()
######################
# Prediction function
######################
def prediction(path):
    # result = get_features(path)
    # prediction = prosody_model.predict(result)
    predicted_class, predicted_probs = emotion2vec.predict(path)
    #y_prediction = prosody_encoder.inverse_transform(prediction.reshape(1, -1))
    #predicted_class = y_prediction[0][0]
    # class probabilities
    #predicted_probs = prediction[0]
    # class names from encoder
    #class_names = prosody_encoder.categories_[0]
    ''''
    # Print predicted class and probabilities for all classes
    print("Predictions for all classes:")
    for label, prob in zip(class_names, predicted_probs):
        print(f"{label}: {prob*100:.2f}%")
    '''
    return predicted_class, predicted_probs


###################
# Database Function
###################
def df_database_function(database_folder):
    
    datagrams = []
    for filename in os.listdir(database_folder):
        if filename.endswith('.wav'):
            emotion = filename.split('_')[0]
            file_path = os.path.join(database_folder, filename)
            datagram = {'path': file_path, 'Emotions': emotion}
            datagrams.append(datagram)
            
    df0 = pd.DataFrame(datagrams)



############################
# Model Fine-Tune 
############################

def get_date_string():
    current_datetime = datetime.now(timezone.utc)

    # Format the datetime as desired
    formatted_datetime = current_datetime.strftime('%Y_%m_%d_%H-%M')
    return formatted_datetime

def get_latest_experiment(experiments_dir = r"tmp"):

    # List all folders in the experiments directory
    experiment_folders = [folder for folder in os.listdir(experiments_dir)]
    if not experiment_folders: #check if empty
        return "exp_" + get_date_string()
    
    # Parse folder names and extract datetime information
    parsed_folders = []
    for folder_name in experiment_folders:
        try:
            folder_datetime = datetime.strptime(folder_name, 'exp_%Y_%m_%d_%H-%M')
            parsed_folders.append((folder_datetime, folder_name))
        except ValueError:
            print(ValueError)
            # Skip folders with names not matching the expected format
            pass

    # Sort the parsed folders based on datetime
    sorted_folders = sorted(parsed_folders, key=lambda x: x[0], reverse=True)

    # Retrieve the latest folder name
    latest_folder = sorted_folders[0][1] if sorted_folders else None

    print("Latest experiment folder:", latest_folder)

    return latest_folder

new_experiment = True



# database_folder = r"./new_recordings"
# df_new = df_database_function(database_folder)


2025-01-15 22:59:16,352 - modelscope - WARNING - Using branch: master as version is unstable, use with caution


Detect model requirements, begin to install it: C:\Users\Alkostantini-GIGA\.cache\modelscope\hub\iic\emotion2vec_plus_large\requirements.txt
install model requirements successfully
ckpt: C:\Users\Alkostantini-GIGA\.cache\modelscope\hub\iic\emotion2vec_plus_large\model.pt
init param, map: modality_encoders.AUDIO.extra_tokens from d2v_model.modality_encoders.AUDIO.extra_tokens in ckpt
init param, map: modality_encoders.AUDIO.alibi_scale from d2v_model.modality_encoders.AUDIO.alibi_scale in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.0.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.weight in ckpt
init param, map: modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias from d2v_model.modality_encoders.AUDIO.local_encoder.conv_layers.0.2.1.bias in ckpt
init param, 

## <h2 style="text-align: left; font-size: 28px; color: #2ecc71; font-weight: bold;">Prosody Real-time & Prosody Active</h2>

In [4]:
####################################
# Prosody Real-Time & Prosody Active
####################################
# Ubdate prosody active Model
###############################
import matplotlib
matplotlib.use('TkAgg')
######################################################################################    

class EmotionPlotter:
    def __init__(self, root, update_callback):
        self.root = root
        self.root.lift()
        self.root.title("Prosody")
        
        frame_color= '#F5F5F5'
        #self.root.configure(bg='#2c3e50')
        self.root.configure(bg=frame_color)
        self.update_callback = update_callback
        self.class_names = ['Noise', 'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
        self.prediction_history = []  # List to store history of predictions



        # Create subplots for the histogram and the valence-arousal map
        self.fig, (self.ax, self.ax_russell) = plt.subplots(1, 2, figsize=(18, 9))  
        # Configure histogram axis
        self.ax.set_title('Emotion Prediction (Histogram)')
        self.ax.set_xlabel('Emotion')
        self.ax.set_ylabel('Probability (%)')
        self.ax.set_ylim(0, 100)
        self.ax.set_xticks(np.arange(len(self.class_names)))
        self.ax.set_xticklabels(self.class_names, rotation=0)

        # Configure valence-arousal Russell map axis
        self.ax_russell.set_title('Valence-Arousal Map (Russell Map)')
        self.ax_russell.set_xlim(-1, 1)
        self.ax_russell.set_ylim(-1, 1)
        self.ax_russell.set_xlabel('Valence')
        self.ax_russell.set_ylabel('Arousal')
        self.ax_russell.axhline(0, color='gray', lw=1)
        self.ax_russell.axvline(0, color='gray', lw=1)

        # self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        # self.canvas.draw()
        # self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=1)



        # # Create a matplotlib figure and axis
        # self.fig, self.ax = plt.subplots()

        # # Embed the plot in the Tkinter window
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.root)
        self.canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

        # Optional: display the chart initially
        self.ax.bar([0, 1, 2], [0, 0, 0])  # Initialize with empty chart
        self.canvas.draw()

        #########################
        # Recording configuration
        #########################
        self.RECORD_SECONDS = 3  # recording duration
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 44100
        self.CHUNK = 1024
        self.CHUNK_SIZE = 1024
        self.OUTPUT_FILE = OUTPUT_FILE
        #######################################################################
        #########################
        # buttons configuration
        ##########################
        # Button styles
        button_font = ('Helvetica', 14, 'bold')
        button_style = {
            'font': button_font,
            'bg': '#2c3e60',  # Green background
            'fg': 'white',    # White text
            'activebackground': '#2c3e55',  # Darker green when pressed
            'activeforeground': 'white',    # White text when pressed
            'borderwidth': 0,
            'padx': 10,
            'pady': 5
        }

        # Create a frame for the buttons
        button_frame = tk.Frame(self.root, bg=frame_color)
        button_frame.pack(side=tk.TOP, pady=(10, 10))

        # Start button
        self.start_button = tk.Button(
            button_frame, text="Start Recording",
            command=self.start_visualization,
            **button_style
        )
        self.start_button.pack(side=tk.LEFT, padx=10, pady=5)

        # Continue button
        self.continue_button = tk.Button(
            button_frame, text="Active Mode",
            command=self.active_mode,
            **button_style
        )
        self.continue_button.pack(side=tk.LEFT, padx=10, pady=5)
        self.continue_button['state'] = tk.DISABLED

        # Exit button
        self.exit_button = tk.Button(
            button_frame, text="Exit",
            command=self.root.destroy,
            **button_style
        )
        self.exit_button.pack(side=tk.RIGHT, padx=10, pady=5)
        self.exit_button['state'] = tk.NORMAL

        # Listen button
        self.listen_button = tk.Button(
            button_frame, text="Play the Voice",
            command=self.play_voice,
            **button_style
        )
        self.listen_button.pack(side=tk.RIGHT, padx=10, pady=5)
        self.listen_button['state'] = tk.DISABLED
        self.visualizing = False                 # track of visualization state
        self.root.after(0, self.center_window)  



    	# # Continuous
        # self.continuous_button = tk.Button(
        #     button_frame, text="Continuous Prediction", 
        #     command=self.start_continuous_prediction,
        #     **button_style
        # )
        # self.continuous_button.pack(side=tk.LEFT, padx=10, pady=5)
        # self.continuous_button['state'] = tk.NORMAL  # Enable the button   

        # Exit button
        # self.exit_button = tk.Button(
        #     button_frame, text="Stop Continuous Mode",
        #     command=self.stop_continuous_mode,
        #     **button_style
        # )
        # self.exit_button.pack(side=tk.RIGHT, padx=10, pady=5)





    def center_window(self):
        # self.root.update_idletasks()  
        # width = self.root.winfo_width()
        # height = self.root.winfo_height()
        # x = (self.root.winfo_screenwidth() // 2) - (width // 2)
        # y = (self.root.winfo_screenheight() // 2) - (height // 2)
        # self.root.geometry(f'{width}x{height}+{x}+{y}')

        self.root.update_idletasks()  # Update "idle" tasks to get the current size.
        # Set fullscreen
        self.root.attributes('-fullscreen', True)
        # Optionally, you can bind 'Esc' to exit fullscreen
        self.root.bind("<Escape>", lambda event: self.root.attributes('-fullscreen', False))
        
    def update_callback(self, predicted_class, predicted_probs):
        """Update the chart based on the predicted class and probabilities."""
        
        # Clear the current plot
        self.ax.clear()

        # Plot updated probabilities
        self.ax.bar(range(len(predicted_probs)), predicted_probs)

        # Update the title or labels as needed
        self.ax.set_title(f'Predicted Class: {predicted_class}')
        self.ax.set_xlabel('Emotion Classes')
        self.ax.set_ylabel('Probability')

        # Redraw the canvas to reflect the changes
        self.canvas.draw()
    ############################################################
    ############################        
    # buttons update
    ##########################
    def start_continuous_prediction(self):
        """Start continuous prediction loop."""
        self.visualizing = True
        self.start_button['state'] = tk.DISABLED
        self.continue_button['state'] = tk.DISABLED
        self.listen_button['state'] = tk.DISABLED
        self.continuous_button['state'] = tk.DISABLED
        self.exit_button['state'] = tk.DISABLED
        self.root.update()
        self.canvas.draw()

        # Start continuous update loop
        self.continuous_prediction_loop()

    def continuous_prediction_loop(self):
        """Loop that handles continuous prediction and chart update."""
        if self.visualizing: 
            # Perform a new prediction
            global PREDICTED_CLASS
            self.root.update()

            PREDICTED_CLASS, predicted_probs = prediction(self.OUTPUT_FILE)
            self.root.update()

            # Update the chart with the latest prediction
            self.update_callback(self, PREDICTED_CLASS, predicted_probs)
            # Trigger redrawing of the chart
            self.canvas.draw()
            self.root.update()

            # Repeat this after 3 seconds 
            self.root.after(3000, self.continuous_prediction_loop)
            self.root.update()

        

    def stop_continuous_mode(self):
        """Stop continuous prediction and enable other buttons."""
        self.visualizing = False  # Stop the continuous prediction loop
        self.start_button['state'] = tk.NORMAL
        self.continue_button['state'] = tk.NORMAL
        self.listen_button['state'] = tk.NORMAL
        self.continuous_button['state'] = tk.NORMAL
        self.exit_button['state'] = tk.NORMAL
        self.exit_button.config(text="Exit")



    def start_visualization(self):
        if not self.visualizing:
            self.visualizing = True
            self.start_button['state'] = tk.DISABLED
            self.continue_button['state'] = tk.DISABLED
            self.listen_button['state'] = tk.DISABLED
            self.update_start_loop()
    
    def active_mode(self):
        if not self.visualizing:
            self.visualizing = True
            self.start_button['state'] = tk.DISABLED
            self.continue_button['state'] = tk.DISABLED
            self.listen_button['state'] = tk.DISABLED
            self.exit_button['state'] = tk.DISABLED
            self.visualizing = False       
        #OUTPUT_FILE=r"./Output/input_voice.wav"
        database_folder= r'./Database'
        self.Prosody_active(self.OUTPUT_FILE, database_folder, self.start_button, self.continue_button, self.listen_button, self.exit_button).active_GUI()
          
    def play_voice(self):

        #playsound(OUTPUT_FILE)
        # sound = AudioSegment.from_wav(OUTPUT_FILE)
        # play(sound)
        # self.root.update()
        sound = AudioSegment.from_wav(self.OUTPUT_FILE)

        audio_data = np.array(sound.get_array_of_samples(), dtype=np.int16)

        # Play the audio using sounddevice
        sd.play(audio_data, samplerate=sound.frame_rate)
        sd.wait()  # Wait until the sound is finished playing
    
    def update_start_text(self, text):
        self.start_button.config(text=text)
        self.root.update()
        
    def update_continue_text(self, text):
        self.continue_button.config(text=text)
        self.root.update()   
             
    def update_start_loop(self):
        ########################
        # Start recording audio
        ########################
        self.update_start_text("Recording in Progress...")
        print("Listening...")
        audio = pyaudio.PyAudio()
        stream = audio.open(format=self.FORMAT, channels=self.CHANNELS, rate=self.RATE, input=True, frames_per_buffer=self.CHUNK)
        frames = []
        # Recording 
        for i in range(0, int(self.RATE / self.CHUNK_SIZE * self.RECORD_SECONDS)):
            data = stream.read(self.CHUNK_SIZE)
            frames.append(data)
        # Stop recording
        stream.stop_stream()
        stream.close()
        audio.terminate()
        # Saving audio
        with wave.open(self.OUTPUT_FILE, 'wb') as wf:
            wf.setnchannels(self.CHANNELS)
            wf.setsampwidth(audio.get_sample_size(self.FORMAT))
            wf.setframerate(self.RATE)
            wf.writeframes(b''.join(frames))
             
        ######################
        # Perform prediction
        ######################
        self.update_start_text("Restart Recording")
        #self.update_continue("active Model")
        self.start_button['state'] = tk.NORMAL
        self.continue_button['state'] = tk.NORMAL
        self.listen_button['state'] = tk.NORMAL
        #######################
        global PREDICTED_CLASS
        #######################
        PREDICTED_CLASS, predicted_probs = prediction(self.OUTPUT_FILE)
        # callback
        self.update_callback(self, PREDICTED_CLASS, predicted_probs)
        # Reset visualization
        self.visualizing = False
        self.root.update() 
    ##################################################################
    ########################
    # Update Plot
    ########################
    def update_plot(self, predicted_probs):

        # Update Histogram
        self.ax.clear()
        bars = self.ax.bar(self.class_names, predicted_probs * 100, color='blue')
        self.ax.set_title('Emotion Prediction (Histogram)')
        self.ax.set_xlabel('Emotion')
        self.ax.set_ylabel('Probability (%)')
        self.ax.set_ylim(0, 100)
        self.ax.set_xticks(np.arange(len(self.class_names)))
        self.ax.set_xticklabels(self.class_names, rotation=0)

        for i, bar in enumerate(bars):
            self.ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 1, f'{predicted_probs[i]*100:.2f}%', ha='center', va='bottom')


        # Update Russell Valence-Arousal Map
        # Russel's map table
        data = {
            "mood": [
                "sleepy & tired", ".", "afraid", "angry", "calm", "relaxed", "content",
                "depressed", "discontent", "determined", "happy", "anxious", "good",
                "pensive", "impressed", "frustrated", "disappointed", "bored", "annoyed",
                "enraged", "excited", "melancholy", "satisfied", "distressed",
                "uncomfortable", "worried", "amused", "apathetic", "peaceful",
                "contemplative", "embarrassed", "sad", "hopeful", "pleased", "None"
            ],
            "valence": [
                0.01, -0.01, -0.12, -0.40, 0.78, 0.71, 0.81, -0.81, -0.68, 0.73, 0.89, -0.72, 0.90,
                0.09, 0.39, -0.60, -0.80, -0.35, -0.44, -0.18, 0.70, -0.05, 0.77, -0.71,
                -0.68, -0.07, 0.55, -0.20, 0.55, 0.58, -0.31, -0.81, 0.61, 0.89, 0.00
            ],
            "arousal": [
                -1.00, -1.00, 0.79, 0.79, -0.68, -0.65, -0.55, -0.48, -0.32, 0.26, 0.17, -0.80, -0.08,
                -0.60, -0.06, 0.40, -0.03, -0.78, 0.76, 0.83, 0.71, -0.65, -0.63, 0.55,
                -0.37, -0.32, 0.19, -0.12, -0.80, -0.60, -0.60, -0.40, -0.30, -0.10, 0.00
            ],
            "#posts": [
                11549, 20308, 20, 3152, 10040, 3545, 11177, 6383, 2490, 3360, 16518, 7039, 5062,
                2322, 1610, 4356, 3328, 12784, 8247, 1016, 11074, 2274, 2629, 2233,
                1763, 3252, 24231, 2732, 2513, 10718, 1092, 6119, 5312, 3517, 3000
            ]
        }
        df = pd.DataFrame(data)

        self.ax_russell.clear()
        self.ax_russell.set_title('Emotion Prediction (Russell Map)')
        self.ax_russell.set_xlim(-1, 1)
        self.ax_russell.set_ylim(-1, 1)
        self.ax_russell.axhline(0, color='gray', lw=1)
        self.ax_russell.axvline(0, color='gray', lw=1)

        emotion_coordinates = {
            "angry": (-0.40, 0.79),
            "disgust": (-0.68, -0.37),  
            "fear": (-0.12, 0.79),     
            "happy": (0.89, 0.17),
            "neutral": (0.00, 0.00),
            "sad": (-0.81, -0.40),
            "surprise": (0.70, 0.71)  
        }


    #         # coordinates for the 7 emotions
    # emotion_coordinates = {
    #     "Noise": (0.00, 0.00),
    #     "angry": (-0.40, 0.79),
    #     "disgust": (-0.68, -0.37),  # uncomfortable
    #     "fear": (-0.12, 0.79),     # afraid
    #     "happy": (0.89, 0.17),
    #     #"neutral": (0.78, -0.68),  # calm
    #     "neutral": (0.00, 0.00),
    #     "sad": (-0.81, -0.40),
    #     "surprise": (0.70, 0.71)   # excited
    # }


        # Calculate weighted valence and arousal
        weighted_valence = 0
        weighted_arousal = 0
        total_percentage = sum(predicted_probs[1:])  # Excluding 'Noise'

        for i, emotion in enumerate(self.class_names[1:]):  # Skip 'Noise'
            valence, arousal = emotion_coordinates.get(emotion, (0, 0))
            weighted_valence += valence * predicted_probs[i + 1]  # Adjust index for skipped 'Noise'
            weighted_arousal += arousal * predicted_probs[i + 1]
        
        weighted_valence /= total_percentage
        weighted_arousal /= total_percentage

        # Store the current prediction
        self.prediction_history.append((weighted_valence, weighted_arousal))


        # Clear current plots
        self.ax_russell.clear()
        self.ax_russell.set_title('Emotion Prediction (Russell Map)')
        self.ax_russell.set_xlim(-1, 1)
        self.ax_russell.set_ylim(-1, 1)
        self.ax_russell.axhline(0, color='gray', lw=1)
        self.ax_russell.axvline(0, color='gray', lw=1)

        # Plot historical points
        for valence, arousal in self.prediction_history:
            self.ax_russell.scatter(valence, arousal, color='orange', s=50, alpha=0.5)


        # Plot the weighted valence-arousal point
        self.ax_russell.scatter(weighted_valence, weighted_arousal, color='red', s=100, label='Predicted Emotion')

        self.ax_russell.scatter(df['valence'], df['arousal'], s=df['#posts'], alpha=0.5, label='Moods')  # Posts the area of the points
        for i, row in df.iterrows():
            plt.text(row['valence'], row['arousal'], row['mood'], fontsize=9, ha='right')

        # # Add emotion coordinates for reference
        # for emotion, (valence, arousal) in emotion_coordinates.items():
        #     self.ax_russell.scatter(valence, arousal, label=emotion, color='orange')
        #     self.ax_russell.text(valence + 0.02, arousal + 0.02, emotion, fontsize=10)

        #self.ax_russell.legend(loc='upper right')
        
        # Redraw canvas
        self.canvas.draw()
   
    #######################################################################################
    #######################################################################################
    # active-learning      
    #######################################################################################
    #######################################################################################
    class Prosody_active:
        def __init__ (self, output_file, output_folder, start_button, continue_button, listen_button,exit_button):
            self.start_button = start_button
            self.continue_button = continue_button
            self.listen_button = listen_button
            self.exit_button = exit_button
            
            self.OUTPUT_FILE = output_file
            self.database_folder = output_folder
            self.predicted_class = PREDICTED_CLASS
            self.classes = ['Noise', 'angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

            self.selected_label = tk.StringVar()
            self.root = tk.Toplevel()               # giving active Gui Root other level as main Gui Root
            self.root.title("Prosody Active-Learning")
            self.root.geometry("700x200")
            self.selected_label.set(None)
            self.root.protocol("WM_DELETE_WINDOW", self.on_window_close)
            self.root.after(0, self.center_window)
            
        def center_window(self):
            self.root.update_idletasks()  
            width = self.root.winfo_width()
            height = self.root.winfo_height()
            x = (self.root.winfo_screenwidth() // 2) - (width // 2)
            y = (self.root.winfo_screenheight() // 2) - (height // 2)
            self.root.geometry(f'{width}x{height}+{x}+{y}')

        def on_window_close(self):
                # Reset button states when window is closed
                self.start_button['state'] = tk.NORMAL
                self.continue_button['state'] = tk.NORMAL
                self.listen_button['state'] = tk.NORMAL
                self.exit_button['state'] = tk.NORMAL
                # Destroy the window
                self.root.destroy()
        #######################
        # Data Folder structure
        #######################       
        def save_data(self, selected_value):    
            if not os.path.exists(self.database_folder):
                os.makedirs(self.database_folder)
                
            existing_files = [f for f in os.listdir(self.database_folder) if f.endswith('.wav')]
            ##########################
            # Find the max file value
            ##########################
            max_number = 0
            for file_name in existing_files:
                match = re.match(rf"{selected_value}_(\d+)\.wav", file_name)
                if match:
                    number = int(match.group(1))
                    max_number = max(max_number, number)
            ###########################   
            # save the file
            ###############
            new_file_name = f"{selected_value}_{max_number + 1}.wav"
            output_file_path = os.path.join(self.database_folder, new_file_name)
            shutil.copyfile(self.OUTPUT_FILE, output_file_path)
            print('saved to Database')
            ############################
            messagebox.showinfo("Success", "Information saved to Prosody Database.")
        #####################################################################################
        ############################
        # Fine Tune
        #############################
        # def fine_tune(self, correct_label):
        
        #     print ('correct_label: ', correct_label)
            
        #     df_database = pd.DataFrame()
        #     df_database = df_database_function(self.database_folder)
        #     df_database.to_csv(r'./Models/df_Database.csv', index = False)
        #     model_finetune(df_database)

        #     messagebox.showinfo("Success", "Model retrained with the new information.")
        #     self.root.destroy()
        #################################################################################            
        ###########################
        # Save without training
        ###########################
        def B_save_data(self):
            selected_value = self.selected_label.get()
            if selected_value in self.classes:
                self.save_data(selected_value)
                print('Data saved without training. Selected label: ', selected_value)

                self.start_button['state'] = tk.NORMAL
                self.continue_button['state'] = tk.NORMAL
                self.listen_button['state'] = tk.NORMAL
                self.exit_button['state'] = tk.NORMAL
                self.visualizing = False
                self.root.update()
                
                if tk._default_root is not None:  # check root window exists
                    self.root.destroy()
        #########################################################################            
        ############################# 
        # Save and Retrain the model
        #############################
        def B_save_retrain(self):
            self.start_button.config(text='Retraining')
            self.continue_button.config(text='in')
            self.listen_button.config(text='progress')
            self.exit_button.config(text='...')
            
            selected_value = self.selected_label.get()
            if selected_value in self.classes:
                self.save_data(selected_value)
                self.fine_tune(selected_value)
                print('Selected label: ', selected_value)
                
                self.start_button['state'] = tk.NORMAL
                self.continue_button['state'] = tk.NORMAL
                self.listen_button['state'] = tk.NORMAL
                self.exit_button['state'] = tk.NORMAL
                
                self.start_button.config(text='Restart Recording')
                self.continue_button.config(text='Active Mode')
                self.listen_button.config(text='Play the Voice')
                self.exit_button.config(text='Exit')
                
                self.visualizing = False
                self.root.update()
                
                if tk._default_root is not None:  # check root window exists
                    self.root.destroy()
        ########################################################################            
        ###########################
        # Bottums & Perform text 
        ###########################
        def active_GUI(self):
            #self.selected_label.set(None)
            predicted_label_text = tk.StringVar()
            predicted_label_text.set("Predicted Class: " + self.predicted_class + "\n Correct the Class")
            predicted_label_label = tk.Label(self.root, textvariable=predicted_label_text, font=("Arial", 16))
            predicted_label_label.pack()
            checkbox_frame = tk.Frame(self.root)
            checkbox_frame.pack(padx=10, pady=10)
            for label in self.classes:
                radio_button = Radiobutton(checkbox_frame, text=label, variable=self.selected_label, value=label,
                                        font=("Arial", 14))
                radio_button.pack(side=tk.LEFT, padx=10)
            save_button = tk.Button(self.root, text="Save to Database", command=self.B_save_data, font=("Arial", 14))
            save_button.pack()

            save_retrain_button = tk.Button(self.root, text="Save to Database and retrain the model",
                                            command=self.B_save_retrain, font=("Arial", 14))
            save_retrain_button.pack()

            self.root.mainloop()
            
            
def visualize_emotion_prediction(plotter, predicted_class, predicted_probs):
    plotter.update_plot(predicted_probs)



In [ ]:
if __name__ == "__main__":
    
    # callback function
    root = tk.Tk() #root window
    ####################################
    # Ensure the window stays on top initially
    root.attributes("-topmost", True)
    root.after(0, lambda: root.attributes("-topmost", False))

    
    # root.lift()
    #####################################
    
    
    plotter = EmotionPlotter(root, visualize_emotion_prediction)
    #plotter = EmotionPlotter(root, update_callback=None)

    root.mainloop()

Listening...


rtf_avg: 0.176: 100%|██████████| 1/1 [00:00<00:00,  1.89it/s]                                                                                      


难过/sad
Listening...


rtf_avg: 0.109: 100%|██████████| 1/1 [00:00<00:00,  3.04it/s]                                                                                      


难过/sad
Listening...


...

rtf_avg: 0.128: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]                                                                                      


难过/sad
Listening...


rtf_avg: 0.108: 100%|██████████| 1/1 [00:00<00:00,  3.06it/s]                                                                                      


难过/sad


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Alkostantini-GIGA\anaconda3\envs\interface\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Alkostantini-GIGA\AppData\Local\Temp\ipykernel_27000\1437577428.py", line 252, in active_mode
    self.Prosody_active(self.OUTPUT_FILE, database_folder, self.start_button, self.continue_button, self.listen_button, self.exit_button).active_GUI()
  File "C:\Users\Alkostantini-GIGA\AppData\Local\Temp\ipykernel_27000\1437577428.py", line 600, in active_GUI
    self.root.mainloop()
  File "c:\Users\Alkostantini-GIGA\anaconda3\envs\interface\lib\tkinter\__init__.py", line 1429, in mainloop
    self.tk.mainloop(n)
KeyboardInterrupt


In [ ]:
df=pd.read_csv(r'.\Models\df_Database.csv')
df